In [ ]:
#Run only at first
!pip install transformers
!pip install torch torchvision torchaudio
!pip install Pillow
!pip install matplotlib

In [11]:
# Core function to convert image to text using mathOCR
import torch
import time
from PIL import Image as PilImage, ImageOps # Renamed Image to PilImage to avoid conflict
from transformers import TrOCRProcessor, VisionEncoderDecoderModel # Note: This still uses TrOCRProcessor/VisionEncoderDecoderModel from Hugging Face

def run_mathOCR(image,
              processor,
              model,
              measure_time: bool = False):
    """
    Converts an image to text using mathOCR.

    Args:
        image (PilImage.Image): Image to be OCR'd.
        processor (TrOCRProcessor): Hugging Face TrOCR Processor (used for mathOCR).
        model (VisionEncoderDecoderModel): Hugging Face TrOCR Model (used for mathOCR).
        measure_time (bool): If True, prints execution time.
    """
    if measure_time:
        t0 = time.perf_counter()
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Process image to pixel values and move to device
    pixel_values = processor(images=image, return_tensors="pt").pixel_values.to(device)

    # Generate text IDs
    generated_ids = model.generate(
        pixel_values,
        max_new_tokens=256,
        num_beams=4,
        early_stopping=True
    )

    # Decode generated IDs to text
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

    if measure_time:
        t1 = time.perf_counter()
        print(f"[Timer] Elapsed: {t1 - t0:0.4f} sec")

    return generated_text

In [12]:
# Function to preprocess the image
def preprocess_image(image_path, target_size=(384, 384)):
    img = PilImage.open(image_path).convert("RGB")
    img = ImageOps.exif_transpose(img)
    img = ImageOps.pad(img, target_size, color="white")
    return img

In [13]:
# --- Main execution flow ---

# Load mathOCR model and processor once
model_name = "fhswf/TrOCR_Math_handwritten" # This is the specific model ID for handwritten math
processor = TrOCRProcessor.from_pretrained(model_name)
model = VisionEncoderDecoderModel.from_pretrained(model_name)

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 16,
  "num_channels": 3,
  "num_hidden_layers": 24,
  "patch_size": 16,
  "pooler_act": "tanh",
  "pooler_output_size": 1024,
  "qkv_bias": false,
  "torch_dtype": "float32",
  "transformers_version": "4.51.3"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_head

In [ ]:
# Replace with your actual image path
image_path = ".jpg your image path"# !!! IMPORTANT: Update this path !!!
img = preprocess_image(image_path)

# Run mathOCR and print the result
extracted_text = run_mathOCR(img, processor, model, measure_time=True) #False if you don't want to display timer
print(extracted_text)


[Timer] Elapsed: 2.1033 sec
e^{i\pi}+1=0.


In [ ]:
# Save and display LaTeX output as an image
from sympy import preview
from IPython.display import Image, display
import os

latex_code = extracted_text
output_path = os.path.join(os.path.expanduser('~'), "latex_output.png")

# Generate the PNG image from LaTeX code
preview(f"${latex_code}$", viewer='file', filename=output_path, euler=False)

# Display the generated image in the output
display(Image(filename=output_path, width=200))